In [4]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from wbfm.utils.xinwei_fdnc.formatting import zimmer2leifer
from wbfm.utils.xinwei_fdnc.predict import load_fdnc_options_and_template, track_using_fdnc
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.auto import tqdm
import os
from fDNC.src.DNC_predict import pre_matt, predict_matches, filter_matches
from wbfm.utils.projects.utils_neuron_names import int2name
from wbfm.utils.feature_detection.utils_candidate_matches import matches_to_sparse_matrix

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from wbfm.utils.projects.finished_project_data import ProjectData
fname = "/scratch/zimmer/Charles/dlc_stacks/worm3-fdnc_pairwise/project_config.yaml"
# fname = "/scratch/zimmer/Charles/dlc_stacks/project-pytest/project_config.yaml"
project_data = ProjectData.load_final_project_data_from_config(fname)

Project data for directory:
/scratch/zimmer/Charles/dlc_stacks/worm3-fdnc_pairwise 
Found the following raw data files:
red_data: True
green_data: True
============Segmentation===============
raw_segmentation: True
segmentation: True
============Tracklets==================
df_training_tracklets: True
reindexed_masks_training: True
============Traces=====================
red_traces: True
green_traces: True
final_tracks: True
behavior_annotations: False



In [6]:
all_matches = project_data.raw_matches

In [7]:
pair = (0,1)
m = all_matches[pair]

In [72]:
project_data.napari_of_single_match(pair)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(15.5, 349.5, 449.5), zoom=1.8524320457796852, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=3, last_used=3, range=((0.0, 1.0, 1.0), (0.0, 31.0, 1.0), (0.0, 699.0, 1.0), (0.0, 899.0, 1.0)), current_step=(0, 0, 0, 0), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'raw_red_data' at 0x7f516190b710>, <Points layer 'n0_zxy_raw' at 0x7f51855013d0>, <Points layer 'n1_zxy_raw' at 0x7f514e88b890>, <Tracks layer 'final_matches' at 0x7f51a8508a90>, <Points layer 'Raw IDs' at 0x7f5155726fd0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, unit=None), text_overlay=TextOverlay(vi

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/charles/anaconda3/envs/torch/lib/python3.7/site-packages/zmq/eventloop/zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "/home/charles/anaconda3/envs/torch/lib/python3.7/site-packages/jupyter_client/threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "/home/charles/anaconda3/envs/torch/lib/python3.7/asyncio/base_events.py", line 563, in run_until_complete
    self._check_runnung()
  File "/home/charles/anaconda3/envs/torch/lib/python3.7/asyncio/base_events.py", line 526, in _check_runnung
    'Cannot run the event loop while another loop is running')
RuntimeError: Cannot run the event loop while another loop is running
ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/charles/anaconda3/envs/torch/lib/pyth

In [62]:
m.options.fdnc_options['topn'] = 3
m.options.affine_num_candidates = 3
m.options.gp_num_candidates = 3

m.match_using_local_affine()
m.match_using_gp()
m.match_using_fdnc()

  0%|          | 0/32 [00:00<?, ?it/s]

In [63]:
m.print_candidates_for_neuron(34)

Candidate: (34, 22, 0.12828506412735577)
Candidate: (34, 38, 0.08218959156929707)
Candidate: (34, 22, 0.11269338491026125)
Candidate: (34, 38, 0.06186985303264858)
Candidate: (34, 18, 0.042727289487510485)
Candidate: (34, 18, 0.25106072)
Candidate: (34, 22, 0.48169032)
Candidate: (34, 27, 0.083414815)
Candidate: (34, 38, 0.61578053)
Candidate: (34, 64, 0.17085364)
Candidate: (34, 150, 0.039738607)


In [73]:
m.print_candidates_for_neuron(17)

Candidate: (17, 17, 0.22438898730022577)
Candidate: (17, 17, 0.08267403811665573)
Candidate: (17, 17, 0.09847124482072768)
Candidate: (17, 4, 0.05902416054054193)
Candidate: (17, 3, 0.047173040897111385)
Candidate: (17, 3, 0.0038649866)
Candidate: (17, 11, 0.0037047665)
Candidate: (17, 17, 0.22010763)
Candidate: (17, 23, 0.03553561)
Candidate: (17, 43, 0.09468804)


In [64]:
m.print_candidates_for_neuron(22, 1)

Candidate: (23, 22, 0.2823131087939511)
Candidate: (23, 22, 0.22357340527827058)
Candidate: (34, 22, 0.12828506412735577)
Candidate: (14, 22, 0.05226103677359605)
Candidate: (23, 22, 0.10217517499405428)
Candidate: (34, 22, 0.11269338491026125)
Candidate: (34, 22, 0.48169032)
Candidate: (23, 22, 0.42206722)
Candidate: (14, 22, 0.07752046)


In [65]:
m.print_candidates_for_neuron(38, 1)

Candidate: (138, 38, 0.31615974186233947)
Candidate: (34, 38, 0.08218959156929707)
Candidate: (23, 38, 0.037147702065015276)
Candidate: (34, 38, 0.06186985303264858)
Candidate: (34, 38, 0.61578053)
Candidate: (23, 38, 0.27889913)
Candidate: (138, 38, 0.058087867)


In [66]:
m.calc_final_matches_using_bipartite_matching();

In [71]:
m.get_f0_to_f1_dict()[34]

38

In [75]:
m.options.min_confidence = 0.4
m.calc_final_matches_using_bipartite_matching();

In [76]:
project_data.napari_of_single_match(pair)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(15.5, 349.5, 449.5), zoom=1.8524320457796852, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=3, last_used=3, range=((0.0, 1.0, 1.0), (0.0, 31.0, 1.0), (0.0, 699.0, 1.0), (0.0, 899.0, 1.0)), current_step=(0, 0, 0, 0), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'raw_red_data' at 0x7f5176d5fbd0>, <Points layer 'n0_zxy_raw' at 0x7f515497ed50>, <Points layer 'n1_zxy_raw' at 0x7f51b4636c50>, <Tracks layer 'final_matches' at 0x7f518daec190>, <Points layer 'Raw IDs' at 0x7f515a030210>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, unit=None), text_overlay=TextOverlay(vi

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/charles/anaconda3/envs/torch/lib/python3.7/site-packages/zmq/eventloop/zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "/home/charles/anaconda3/envs/torch/lib/python3.7/site-packages/jupyter_client/threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "/home/charles/anaconda3/envs/torch/lib/python3.7/asyncio/base_events.py", line 563, in run_until_complete
    self._check_runnung()
  File "/home/charles/anaconda3/envs/torch/lib/python3.7/asyncio/base_events.py", line 526, in _check_runnung
    'Cannot run the event loop while another loop is running')
RuntimeError: Cannot run the event loop while another loop is running
ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/charles/anaconda3/envs/torch/lib/pyth